## Toma de radicados de salida

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains
import requests
import lxml
import time
from time import sleep
from selenium.webdriver.chrome.options import Options
import email
import email.parser
from email import policy
from email.parser import BytesParser
from email import message_from_file
import os
import glob
from multiprocessing import Pool
import requests
import urllib3
import pandas as pd
import os
from datetime import datetime
import numpy as np
import xlrd
from os import remove
import re
import urllib.parse
from email.header import decode_header
import shutil
import sys
import smtplib
import time
import imaplib
import email
import traceback

## Se abre base de datos

In [ ]:
base = pd.read_excel(r"C:\Users\jkp28\OneDrive\Consultorias\UPME\Documentos\Bases de datos\Radicados.xlsx",converters={'Radicado entrada':str})
num_radicado = base['Radicadoentrada'].dropna()
radicados = pd.DataFrame(num_radicado)
radicados.insert(1, "Radicado salida", "")
radicados.insert(2, "Fecha respuesta", "")
radicados

## Se ingresa a Orfeo y se toman los radicado

In [ ]:
for i in range(5):
    
    try:
        radicado = radicados['Radicadoentrada'][i]

        # Se define la url y se accede a ella
        url = 'https://orfeo.upme.gov.co/login.php'
        options = webdriver.ChromeOptions()
        options.add_argument('--incognito')
        driver1 = webdriver.Chrome(options=options)
        driver1.get(url)

        # Poner usuario y contraseña para acceder
        username = driver1.find_element("name", "krd")
        username.send_keys('jperdomo')

        password = driver1.find_element("name","drd")
        password.send_keys('Upme2023*')

        sleep(2)
        Consulta = driver1.find_element("xpath", '//*[@class="btn btn-login"]')
        Consulta.click()

        sleep(2)
        poner_token = driver1.find_element("name", 'token')

        url2="https://mail.google.com/mail/u/0/"
        options = webdriver.ChromeOptions()
        options.add_argument('--incognito')
        driver2 = webdriver.Chrome(options=options)
        driver2.get(url2)
        email_phone = driver2.find_element("xpath", "//input[@id='identifierId']")
        email_phone.send_keys("juan.perdomo@upme.gov.co")
        time.sleep(2)
        driver2.find_element("xpath", '//*[@class="FliLIb FmFZVc"]').click()
        password = WebDriverWait(driver2, 10).until(
            EC.element_to_be_clickable(("xpath", '//*[@name="Passwd"]'))
        )
        password.send_keys("Juancamilo-5050")
        driver2.find_element("xpath", '//*[@class="F9NWFb"]').click()
        WebDriverWait(driver2, 5).until(EC.element_to_be_clickable(("xpath", '//*[@class="yW"]'))).click()
        time.sleep(3)
        token = (driver2.find_element("xpath", '//div[@class="a3s aiL "]').text)[-6:]
        driver2.close()

        poner_token.send_keys(token)
        validar = driver1.find_element("xpath", '//*[@class="btn btn-login"]')
        validar.click()

        Bandejas = driver1.find_element("xpath", '//li[@id="bandejas"]')
        Bandejas.click()
        time.sleep(1)
        #Consulta = WebDriverWait(driver1, 3).until(EC.presence_of_all_elements_located((By.XPATH, "//li[@class='dropdown-submenu']//*[@class='menu_princ' and contains(., 'Consultas')]")))
        Consulta = driver1.find_element("xpath", "//li[@class='dropdown-submenu']//a[@class='menu_princ' and contains(., 'Consultas')]")
        time.sleep(2)

        ActionChains(driver1).move_to_element(Consulta).perform()
        Consulta_clasica = driver1.find_element("xpath", "//ul[@class='dropdown-menu']//*[@class='menu_princ' and contains(., 'Consulta Clasica')]").click()

        iframe = driver1.find_elements('xpath', "//iframe[@id='mainFrame']")[0]
        driver1.switch_to.frame(iframe)
        NumRadicado = driver1.find_element("xpath", "//input[@name='s_RADI_NOMB']")
        NumRadicado.send_keys(radicado)

        iframes = driver1.find_elements("xpath", "//*[@target='mainFrame']")
        for iframe in iframes:
            if 'ontouchmove' in iframe.get_attribute("target"):
                driver.switch_to.frame(iframe)

        ano = driver1.find_element("xpath","//*[@name='s_desde_ano']")
        ano.send_keys('2020')

        Boton_buscar = driver1.find_element("xpath","//*[@name='Busqueda']")
        Boton_buscar.click()

        datos = driver1.find_elements("xpath","//*[@class='leidos']//*[@class='vinculos']")
        for dato in datos:
            print(dato.text)

        Respuesta = []
        datos = driver1.find_elements("xpath","//*[@class='leidos']//*[@class='vinculos']")
        for dato in datos:
            Respuesta.append(dato.text)
        Respuesta
        
        radicados["Radicado salida"][i] = Respuesta[0]
        radicados["Fecha respuesta"][i] = Respuesta[1]
        
        driver1.close()
        
    except:
        print("No se encontró el radicado " + radicado)
        
